In [1]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
import pygame
import datetime
import imutils
import time
import dlib
import cv2

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/amit/driver1/facial-landmarks/shape_predictor_68_face_landmarks.dat')

In [ ]:
vs = VideoStream(0).start()
time.sleep(2.0)

COUNTER = 0
while True:

    frame = vs.read()
    time.sleep(1.0)
    pygame.mixer.init()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    
    def sound_alarm_on(path):
        pygame.mixer.music.load(path)
        pygame.mixer.music.play(0)
        
    def sound_alarm_off():
        pygame.mixer.music.stop()
        
    def eye_aspect_ratio(eye):

        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear
    
    EYE_AR_THRESH = 0.2
    EYE_AR_CONSEC_FRAMES = 10
  
    ALARM_ON = False
    
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
 

        ear = (leftEAR + rightEAR) / 2.0
        if ear < EYE_AR_THRESH:
            COUNTER = COUNTER+1
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if not ALARM_ON:
                    ALARM_ON = True
                    sound_alarm_on('Tornado_Siren_II-Delilah-747233690.mp3')
        else:
            COUNTER = 0
            ALARM_ON = False
            sound_alarm_off()
            
    cv2.imshow("Face", frame)
    key = cv2.waitKey(1) & 0xFF
    
if (key == ord("q")):
    break  
        
        
cv2.destroyAllWindows()
vs.stop()            